# 20240530 筆記

## 作業

請使用 YOLO 根據 kaggle 上面的 traffic sign detection 資料集，建立一個自己的 model。

https://www.kaggle.com/datasets/pkdarabi/cardetection

## 從 image 裡面取得 foreground object

In [8]:
import cv2
import numpy as np
import plotly.express as px

In [2]:
import plotly.io as pio
pio.renderers.default = 'iframe'

In [5]:
image = cv2.imread('metal100.jpg')

In [7]:
px.imshow(image[:,:,::-1])

In [9]:
# 建立 mask
mask = np.zeros(image.shape[:2], np.uint8)

# 定義模型
bgdModel = np.zeros((1, 65), np.float64)
fgdModel = np.zeros((1, 65), np.float64)

# 繪製矩形，包含前景對象
rect = (60, 10, 440, 350)

# 應用GrabCut算法
cv2.grabCut(image, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)

# 生成前景掩碼
mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
foreground = image * mask2[:, :, np.newaxis]

In [10]:
px.imshow(foreground[:,:,::-1])

In [11]:
import gradio as gr

In [12]:
def process(image, xmin, ymin, xmax, ymax):
    # 建立 mask
    mask = np.zeros(image.shape[:2], np.uint8)
    
    # 定義模型
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)
    
    # 繪製矩形，包含前景對象
    rect = (xmin, ymin, xmax, ymax)
    
    # 應用GrabCut算法
    cv2.grabCut(image, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)
    
    # 生成前景掩碼
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    foreground = image * mask2[:, :, np.newaxis]
    return foreground

In [13]:
gr.Interface(process,
             ["image", "number", "number", "number", "number"],
             "image").launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


## 使用 YOLOv8 的 Instance Segmentation

In [14]:
from ultralytics import YOLO

In [15]:
model = YOLO("yolov8n-seg.pt")

100%|█████████████████████████████████| 6.73M/6.73M [00:01<00:00, 7.01MB/s]


In [16]:
results = model('therock.jpg')


image 1/1 C:\Users\Victor\vcdemy\20240530\therock.jpg: 480x640 1 person, 1 tie, 1 chair, 686.0ms
Speed: 12.5ms preprocess, 686.0ms inference, 2036.8ms postprocess per image at shape (1, 3, 480, 640)


In [17]:
results[0].show()

## Instance Segmentation 只畫出 polygon 的形狀。

In [18]:
image = cv2.imread('therock.jpg')

In [19]:
image = image[:,:,::-1]

In [20]:
results = model(image)


0: 480x640 1 person, 1 tie, 359.1ms
Speed: 10.2ms preprocess, 359.1ms inference, 26.1ms postprocess per image at shape (1, 3, 480, 640)


In [23]:
image.dtype

dtype('uint8')

In [21]:
for result in results:
    for seg in result.masks.xy:  # 獲取分割多邊形
        # 轉換為整數類型座標
        points = np.array(seg, dtype=np.int32)
        # 繪製多邊形
        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)

error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'polylines'
> Overload resolution failed:
>  - Layout of the output array img is incompatible with cv::Mat
>  - Expected Ptr<cv::UMat> for argument 'img'
